<a href="https://colab.research.google.com/github/mrsanap/AICS_LAB/blob/main/spam_filter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

01: Spam Filter Using Naive Bayes Algorithm
Sms spam classifier

Data Reading and Cleaning,
EDA (Exploratory Data Analysis),
Model Building,
Evolution,
Improvement,
Deploy

In [2]:

#import rquired libraies for reading and cleaning
import nltk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer,WordNetLemmatizer
from nltk.tokenize import word_tokenize
import sklearn.metrics as m


In [4]:
#Download and unzip some subpackeges of nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
     

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [9]:

#read the dataset
df = pd.read_csv('spam.csv', encoding='latin-1')

In [11]:

#check the first five row
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [13]:

#check the shape of the data set
df.shape

(5572, 5)

In [15]:

sent = dataset.iloc[:, [1]]['v2']
sent

0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
                              ...                        
5567    This is the 2nd time we have tried 2 contact u...
5568                Will  _ b going to esplanade fr home?
5569    Pity, * was in mood for that. So...any other s...
5570    The guy did some bitching but I acted like i'd...
5571                           Rofl. Its true to its name
Name: v2, Length: 5572, dtype: object

In [17]:

labels = dataset.iloc[:, [0]]['v1']
labels

0        ham
1        ham
2       spam
3        ham
4        ham
        ... 
5567    spam
5568     ham
5569     ham
5570     ham
5571     ham
Name: v1, Length: 5572, dtype: object

In [19]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
labels = le.fit_transform(labels)

In [26]:
le.classes_


array([0, 1])

In [28]:

labels

array([0, 0, 1, ..., 0, 0, 0])

In [29]:

from keras.utils.np_utils import to_categorical
labels = to_categorical(labels)

In [31]:
labels

array([[1., 0.],
       [1., 0.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [35]:
import re

Next, we will process the message content with Regular Expressions (Regex) to keep email and web addresses, phone numbers, and numbers uniform, encode symbols, remove punctuation and white spaces, and finally convert all text to lowercase:

In [37]:
lemma = WordNetLemmatizer()

In [39]:
sentence = []
for j in range(0, len(sent)):
  s = re.sub('[^A-Za-z]', ' ', sent[j])
  s = s.lower()
  words = word_tokenize(s)
  words = [lemma.lemmatize(i) for i in words if i not in stopwords.words('english')]
  s = ' '.join(words)
  sentence.append(s)

In [41]:
#sentence

from keras.preprocessing.text import one_hot


In [44]:
one_hot_sent = [one_hot(i, 1000) for i in sentence]
maxi = [len(i) for i in one_hot_sent]
max(maxi)

77

In [46]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [48]:

padsequences = pad_sequences(one_hot_sent, maxlen=80)
padsequences

array([[  0,   0,   0, ..., 915, 717, 444],
       [  0,   0,   0, ..., 167, 716, 594],
       [  0,   0,   0, ..., 365,  79, 609],
       ...,
       [  0,   0,   0, ..., 641, 278, 523],
       [  0,   0,   0, ..., 611, 716, 187],
       [  0,   0,   0, ..., 287, 969, 934]], dtype=int32)

In [51]:
feature_train, feature_test, label_train, label_test = train_test_split(padsequences, labels, test_size=0.2, random_state=7)

In [53]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout, Flatten

In [55]:

model = Sequential()
model.add(Embedding(10000, 64, input_length=80))
model.add(LSTM(100))
model.add(Dense(2, activation='sigmoid'))
model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='adam')

In [57]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 80, 64)            640000    
                                                                 
 lstm_1 (LSTM)               (None, 100)               66000     
                                                                 
 dense_1 (Dense)             (None, 2)                 202       
                                                                 
Total params: 706,202
Trainable params: 706,202
Non-trainable params: 0
_________________________________________________________________


In [59]:
model.fit(feature_train, label_train, epochs=18, validation_data=(feature_test, label_test))

Epoch 1/18
140/140 [==============================] - 8s 60ms/step - loss: 2.0671e-04 - accuracy: 1.0000 - val_loss: 0.2085 - val_accuracy: 0.9632
Epoch 2/18
140/140 [==============================] - 8s 55ms/step - loss: 1.4791e-04 - accuracy: 1.0000 - val_loss: 0.2037 - val_accuracy: 0.9668
Epoch 3/18
140/140 [==============================] - 8s 59ms/step - loss: 1.1610e-04 - accuracy: 1.0000 - val_loss: 0.2126 - val_accuracy: 0.9677
Epoch 4/18
140/140 [==============================] - 8s 58ms/step - loss: 8.3545e-05 - accuracy: 1.0000 - val_loss: 0.2124 - val_accuracy: 0.9668
Epoch 5/18
140/140 [==============================] - 7s 53ms/step - loss: 7.6926e-05 - accuracy: 1.0000 - val_loss: 0.2190 - val_accuracy: 0.9668
Epoch 6/18
140/140 [==============================] - 8s 59ms/step - loss: 6.3314e-05 - accuracy: 1.0000 - val_loss: 0.2184 - val_accuracy: 0.9677
Epoch 7/18
140/140 [==============================] - 8s 59ms/step - loss: 5.0358e-05 - accuracy: 1.0000 - val_loss: 0

In [61]:
label_pred = model.predict(feature_test)
label_pred


35/35 [==============================] - 1s 25ms/step


array([[9.9999976e-01, 2.2401019e-07],
       [9.9994832e-01, 5.3152220e-05],
       [9.9999982e-01, 1.7927296e-07],
       ...,
       [9.9999940e-01, 7.1087482e-07],
       [9.9999976e-01, 3.1563607e-07],
       [9.9999982e-01, 1.6734255e-07]], dtype=float32)

In [62]:
label_pred_ = [np.argmax(i, axis=0) for i in label_pred]
label_pred_[0:5]

[0, 0, 0, 0, 0]

In [63]:
label_test_= [np.argmax(i, axis=0) for i in label_test]
label_test_[0:5]

[0, 0, 0, 0, 0]

In [64]:
m.accuracy_score(label_test_, label_pred_)

0.9668161434977578

In [65]:
print(m.classification_report(label_test_, label_pred_))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98       970
           1       0.89      0.86      0.87       145

    accuracy                           0.97      1115
   macro avg       0.93      0.92      0.93      1115
weighted avg       0.97      0.97      0.97      1115



In [66]:
print(m.confusion_matrix(label_test_, label_pred_))

[[954  16]
 [ 21 124]]
